##### Load the nb_black formatter
pip install nb_black or conda install nb_black -- if not installed already

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

#### Load the required libraries

In [3]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow import keras

# Importing specific modules from libraries
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Importing User-defined functions from utilis
from main_utilis import read_data, data_from_to_numpy, std_normalise_data
from utils import augment_data, standard_model
from equi_nn_utilis import (
    EquivariantHiddenLayer,
    EquivariantOutputLayer,
    eq_build_model,
)

# Setting numpy print options to display floating point numbers up to 3 decimal points
np.set_printoptions(precision=3, suppress=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1. Load the datasets

leakage dataset train [m].csv 

leakage dataset validation 1000.csv

and store the data as NumPy-Arrays
X train, Y train
X validation, Y validation.

#### Loading the data from CSV files

In [4]:
df_100, df_1000, df_val = read_data()

<IPython.core.display.Javascript object>

#### Load the features and label from dataframe and convert it to numpy array

In [5]:
X_train_100, y_train_100, X_train_1000, y_train_1000, X_val, y_val = data_from_to_numpy(
    df_100, df_1000, df_val
)

<IPython.core.display.Javascript object>

####  Standardize and normalize the data
For that purpose, you have training data (xi, yi)
mi=1 available. Unfortunately,
the fourth sensor MFC4 had a slight malfunction at the time when the training data were produced. As a consequence, the values x4 are on average
higher than their counterparts x1, x2 and x3. Be aware that the same effect
is not present in the validation and test data.

In [6]:
X_train_100_norm, X_train_1000_norm, X_val_norm = std_normalise_data(
    X_train_100, X_train_1000, X_val
)

<IPython.core.display.Javascript object>

### Train a standard fully connected neural network without hypertuning

In [ ]:
model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Dense(
        64,
        activation="relu",
        input_shape=(4,),
    )
)
model.add(tf.keras.layers.Dense(32, activation="relu"))

model.add(tf.keras.layers.Dense(2))
# model.add(tf.keras.layers.Dense(2, activation="softmax"))
#

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="mean_absolute_error",
    metrics=["accuracy", "mean_absolute_error"],
)
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
# Train the model
history = model.fit(
    X_train_1000_norm,
    y_train_1000,
    validation_data=(X_val_norm, y_val),
    batch_size=32,
    epochs=100,
    callbacks=[callback],
)

In [ ]:
model.summary()

### 2. Train a standard fully connected neural network. Hyperparameters

like number of epochs, batch size, optimizer, learning rate, depth and
width of the network, activation and loss functions – all up to you. You
can use a framework like KerasTuner to optimize hyperparameters, or
simple for-loops if you prefer that. Anyway, you should use the validation data in order to evaluate and compare candidate hyperparameter
configurations. Save the model as model standard [m].h5.

### For Normalised Dataset with 100 samples

#### Instantiate the tuner and define search space

In [ ]:
# Instantiate the tuner and define search space
tuner_1 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="standard_model_100",
)

In [ ]:
# Define early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10)
# Define the search strategy
tuner_1.search(
    X_train_100_norm,
    y_train_100,
    epochs=200,
    validation_data=(X_val_norm, y_val),
    callbacks=[early_stop],
)

In [ ]:
best_hps = tuner_1.get_best_hyperparameters(num_trials=1)[0]
model_hp_100 = tuner_1.hypermodel.build(best_hps)


In [ ]:
best_hps.values

In [ ]:
model_hp_100.summary()

#### Calling Hyperparameters in the required model

In [ ]:
# Define the input shape
input_shape = (X_train_1000_norm.shape[1],)

# Define the model architecture using the hyperparameters
model_100 = keras.Sequential()
model_100.add(Dense(units=best_hps.values['input_units'], activation=best_hps.values['input_activation'], input_shape=input_shape))
for i in range(best_hps.values['num_layers']):
    model_100.add(Dense(units=best_hps.values[f'hidden_{i+1}_units'], activation=best_hps.values[f'hidden_{i+1}_activation']))
model_100.add(Dense(units=2, activation='linear'))

# Compile the model using the hyperparameters
optimizer = (tf.keras.optimizers.Adam(learning_rate=best_hps.values['learning_rate']))
model_100.compile(optimizer=optimizer, loss=best_hps.values['loss'], metrics=['mae'])
early_stop = EarlyStopping(monitor="val_loss", patience=10)

# Fit the model using the hyperparameters
history_100 = model_100.fit(X_train_100_norm, y_train_100, batch_size=best_hps.values['batch_size'], epochs=best_hps.values['num_epochs'],callbacks=[early_stop], validation_data=(X_val_norm, y_val))


In [ ]:
model_100.summary()

In [ ]:
# # Save the model
model_100.save("models/model_standard_100.h5")

In [ ]:
# Load and Evaluate the model on the validation dataset
new_model_100 = tf.keras.models.load_model("models/model_standard_100.h5")
val_metrics_100 = new_model_100.evaluate(X_val_norm, y_val)

In [ ]:
new_model_100.summary()

### For Normalised Dataset with 1000 samples

In [ ]:
# Instantiate the tuner and define search space
tuner_2 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="standard_model_1000",
)


# Define early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10)
# Define the search strategy
tuner_2.search(
    X_train_1000_norm,
    y_train_1000,
    epochs=200,
    validation_data=(X_val_norm, y_val),
    callbacks=[early_stop],
)

In [ ]:
best_hps_1000 = tuner_2.get_best_hyperparameters(num_trials=1)[0]
model_hp_1000 = tuner_2.hypermodel.build(best_hps_1000)


In [ ]:
best_hps_1000.values

In [ ]:
model_hp_1000.summary()

In [ ]:
# Define the input shape
input_shape = (X_train_1000_norm.shape[1],)

# Define the model architecture using the hyperparameters
model_1000 = keras.Sequential()
model_1000.add(Dense(units=best_hps_1000.values['input_units'], activation=best_hps_1000.values['input_activation'], input_shape=input_shape))
for i in range(best_hps_1000.values['num_layers']):
    model_1000.add(Dense(units=best_hps_1000.values[f'hidden_{i+1}_units'], activation=best_hps_1000.values[f'hidden_{i+1}_activation']))
model_1000.add(Dense(units=2, activation='linear'))

# Compile the model using the hyperparameters
optimizer = (tf.keras.optimizers.Adam(learning_rate=best_hps_1000.values['learning_rate']))
model_1000.compile(optimizer=optimizer, loss=best_hps_1000.values['loss'], metrics=['mae'])
early_stop = EarlyStopping(monitor="val_loss", patience=10)
# Fit the model using the hyperparameters
history_1000 = model_1000.fit(X_train_1000_norm, y_train_1000, batch_size=best_hps_1000.values['batch_size'], epochs=best_hps_1000.values['num_epochs'],callbacks=[early_stop] ,validation_data=(X_val_norm, y_val))


In [ ]:
# # Save the model
model_1000.save("models/model_standard_1000.h5")

In [ ]:
# Load and Evaluate the model on the validation dataset
new_model_1000 = tf.keras.models.load_model("models/model_standard_1000.h5")
val_metrics_1000 = new_model_1000.evaluate(X_val_norm, y_val)

In [ ]:
new_model_1000.summary()

### 3. Augment the dataset. 

Each training example (x, y) can be complemented by seven additional virtual examples as illustrated in Figure 1.
Clockwise rotations and flips on input data x are obtained as described
above. A clockwise rotation on output data y is obtained through
y90 = (y2, −y1) and a flip along the vertical axis on the same data is
represented by yf lipped = (−y1, y2). All other operations (rotation angles and associated flips) can be computed by subsequent application
of these two operations.

#### Function to augment the data and insert into training samples

In [7]:
# Augment the training data with the augment_data function for the training
x_train_augmented, y_train_augmented = augment_data(X_train_100_norm, y_train_100)
x_train_augmented_1000, y_train_augmented_1000 = augment_data(
    X_train_1000_norm, y_train_1000
)


<IPython.core.display.Javascript object>

In [8]:
# Convert the augmented data and labels to numpy arrays
X_train_augmented = np.array(x_train_augmented)
y_train_augmented = np.array(y_train_augmented)
X_train_augmented_1000 = np.array(x_train_augmented_1000)
y_train_augmented_1000 = np.array(y_train_augmented_1000)

<IPython.core.display.Javascript object>

In [8]:
model_1 = tf.keras.Sequential()
model_1.add(
    tf.keras.layers.Dense(
        128,
        activation="relu",
        input_shape=(4,),
    )
)
model_1.add(tf.keras.layers.Dense(32, activation="relu"))

model_1.add(tf.keras.layers.Dense(2, activation="linear"))

# Compile the model
model_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="mean_absolute_error",
    metrics=["accuracy", "mean_absolute_error"],
)
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
# Train the model
history = model_1.fit(
    X_train_augmented_1000,
    y_train_augmented_1000,
    validation_data=(X_val_norm, y_val),
    batch_size=32,
    epochs=100,
    callbacks=[callback],
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5066 - accuracy: 0.4951 - mean_absolute_error: 0.5066 - val_loss: 0.5006 - val_accuracy: 0.4960 - val_mean_absolute_error: 0.5006
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5057 - accuracy: 0.4942 - mean_absolute_error: 0.5057 - val_loss: 0.5003 - val_accuracy: 0.5040 - val_mean_absolute_error: 0.5003
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5057 - accuracy: 0.5002 - mean_absolute_error: 0.5057 - val_loss: 0.5013 - val_accuracy: 0.5040 - val_mean_absolute_error: 0.5013
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5059 - accuracy: 0.4967 - mean_absolute_error: 0.5059 - val_loss: 0.5003 - val_accuracy: 0.5040 - val_mean_absolute_error: 0.5003
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5057 - accuracy: 0.4888 - mean_absolute_error: 0.5057 - val_loss: 0.5005 - val_accuracy: 0.4960

<IPython.core.display.Javascript object>

### For Augmeted Dataset after normalisation with 100 samples

In [9]:
# Instantiate the tuner and define search space
tuner_3 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="standard_model_100_augmented",
)


# Define early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10)
# Define the search strategy
tuner_3.search(
    X_train_augmented,
    y_train_augmented,
    epochs=200,
    validation_data=(X_val_norm, y_val),
    callbacks=[early_stop],
)

Trial 254 Complete [00h 00m 10s]
val_loss: 0.5697615742683411

Best val_loss So Far: 0.5594862103462219
Total elapsed time: 00h 16m 29s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [10]:
best_hps_100_aug = tuner_3.get_best_hyperparameters(num_trials=1)[0]
model_hp_100_aug = tuner_3.hypermodel.build(best_hps_100_aug)


<IPython.core.display.Javascript object>

In [11]:
best_hps_100_aug.values

{'num_layers': 1,
 'layer_size_range': 496,
 'input_units': 64,
 'input_activation': 'tanh',
 'hidden_1_units': 32,
 'hidden_1_activation': 'tanh',
 'optimizer': 'adam',
 'learning_rate': 0.0001,
 'loss': 'mean_squared_error',
 'batch_size': 48,
 'num_epochs': 70,
 'hidden_2_units': 64,
 'hidden_2_activation': 'tanh',
 'hidden_3_units': 64,
 'hidden_3_activation': 'sigmoid',
 'hidden_4_units': 32,
 'hidden_4_activation': 'relu',
 'tuner/epochs': 23,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

<IPython.core.display.Javascript object>

In [12]:
model_hp_100_aug.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                320       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [13]:
# Define the input shape
input_shape = (X_train_augmented_1000.shape[1],)

# Define the model architecture using the hyperparameters
model_100_aug = keras.Sequential()
model_100_aug.add(Dense(units=best_hps_100_aug.values['input_units'], activation=best_hps_100_aug.values['input_activation'], input_shape=input_shape))
for i in range(best_hps_100_aug.values['num_layers']):
    model_100_aug.add(Dense(units=best_hps_100_aug.values[f'hidden_{i+1}_units'], activation=best_hps_100_aug.values[f'hidden_{i+1}_activation']))
model_100_aug.add(Dense(units=2, activation='linear'))

# Compile the model using the hyperparameters
optimizer = (tf.keras.optimizers.Adam(learning_rate=best_hps_100_aug.values['learning_rate']))
model_100_aug.compile(optimizer=optimizer, loss=best_hps_100_aug.values['loss'], metrics=['mae'])
early_stop = EarlyStopping(monitor="val_loss", patience=10)

# Fit the model using the hyperparameters
history_100_aug = model_100_aug.fit(X_train_augmented, y_train_augmented, batch_size=best_hps_100_aug.values['batch_size'], epochs=best_hps_100_aug.values['num_epochs'],callbacks=[early_stop], validation_data=(X_val_norm, y_val))


Epoch 1/70
17/17 [==============================] - 1s 11ms/step - loss: 0.3508 - mae: 0.5107 - val_loss: 0.3519 - val_mae: 0.5142
Epoch 2/70
17/17 [==============================] - 0s 4ms/step - loss: 0.3479 - mae: 0.5092 - val_loss: 0.3483 - val_mae: 0.5119
Epoch 3/70
17/17 [==============================] - 0s 4ms/step - loss: 0.3472 - mae: 0.5091 - val_loss: 0.3470 - val_mae: 0.5110
Epoch 4/70
17/17 [==============================] - 0s 4ms/step - loss: 0.3468 - mae: 0.5089 - val_loss: 0.3456 - val_mae: 0.5101
Epoch 5/70
17/17 [==============================] - 0s 4ms/step - loss: 0.3465 - mae: 0.5088 - val_loss: 0.3448 - val_mae: 0.5096
Epoch 6/70
17/17 [==============================] - 0s 3ms/step - loss: 0.3466 - mae: 0.5090 - val_loss: 0.3434 - val_mae: 0.5086
Epoch 7/70
17/17 [==============================] - 0s 3ms/step - loss: 0.3461 - mae: 0.5087 - val_loss: 0.3407 - val_mae: 0.5067
Epoch 8/70
17/17 [==============================] - 0s 4ms/step - loss: 0.3462 - mae: 0.5

<IPython.core.display.Javascript object>

In [14]:
model_100_aug.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                320       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 2)                 66        
                                                                 
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [15]:
# Save the Model
model_100_aug.save("models/model_standard_100_augmented.h5")

<IPython.core.display.Javascript object>

In [16]:
# Load and Evaluate the model on the validation dataset
new_model_100_aug = tf.keras.models.load_model("models/model_standard_100_augmented.h5")

val_metrics_100_aug = new_model_100_aug.evaluate(X_val_norm, y_val)



32/32 [==============================] - 0s 903us/step - loss: 0.3318 - mae: 0.5008


<IPython.core.display.Javascript object>

In [17]:
new_model_100_aug.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                320       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 2)                 66        
                                                                 
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

### For Augmeted Dataset after normalisation with 1000 samples

In [18]:
# Instantiate the tuner and define search space
tuner_4 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="standard_model_1000_augmented",
)

# Define early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10)
# Define the search strategy
tuner_4.search(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=200,
    validation_data=(X_val_norm, y_val),
    callbacks=[early_stop],
)

Trial 254 Complete [00h 00m 21s]
val_loss: 0.5700260400772095

Best val_loss So Far: 0.5531229376792908
Total elapsed time: 00h 31m 30s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [19]:
best_hps_1000_aug = tuner_4.get_best_hyperparameters(num_trials=1)[0]
model_hp_1000_aug = tuner_4.hypermodel.build(best_hps_1000_aug)


<IPython.core.display.Javascript object>

In [20]:
best_hps_1000_aug.values

{'num_layers': 1,
 'layer_size_range': 80,
 'input_units': 64,
 'input_activation': 'tanh',
 'hidden_1_units': 48,
 'hidden_1_activation': 'tanh',
 'optimizer': 'adam',
 'learning_rate': 0.01,
 'loss': 'mean_squared_error',
 'batch_size': 112,
 'num_epochs': 170,
 'hidden_2_units': 144,
 'hidden_2_activation': 'tanh',
 'hidden_3_units': 992,
 'hidden_3_activation': 'sigmoid',
 'hidden_4_units': 272,
 'hidden_4_activation': 'relu',
 'tuner/epochs': 23,
 'tuner/initial_epoch': 8,
 'tuner/bracket': 4,
 'tuner/round': 2,
 'tuner/trial_id': '0111'}

<IPython.core.display.Javascript object>

In [21]:
model_hp_1000_aug.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                320       
                                                                 
 dense_7 (Dense)             (None, 48)                3120      
                                                                 
 dense_8 (Dense)             (None, 2)                 98        
                                                                 
Total params: 3,538
Trainable params: 3,538
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [22]:
# Define the input shape
input_shape = (X_train_augmented_1000.shape[1],)

# Define the model architecture using the hyperparameters
model_1000_aug = keras.Sequential()
model_1000_aug.add(Dense(units=best_hps_1000_aug.values['input_units'], activation=best_hps_1000_aug.values['input_activation'], input_shape=input_shape))
for i in range(best_hps_1000_aug.values['num_layers']):
    model_1000_aug.add(Dense(units=best_hps_1000_aug.values[f'hidden_{i+1}_units'], activation=best_hps_1000_aug.values[f'hidden_{i+1}_activation']))
model_1000_aug.add(Dense(units=2, activation='linear'))

# Compile the model using the hyperparameters
optimizer = (tf.keras.optimizers.Adam(learning_rate=best_hps_1000_aug.values['learning_rate']))
model_1000_aug.compile(optimizer=optimizer, loss=best_hps_1000_aug.values['loss'], metrics=['mae'])
early_stop = EarlyStopping(monitor="val_loss", patience=10)

# Fit the model using the hyperparameters
history_1000_aug = model_1000_aug.fit(X_train_augmented_1000, y_train_augmented_1000, batch_size=best_hps_1000_aug.values['batch_size'], epochs=best_hps_1000_aug.values['num_epochs'],callbacks=[early_stop], validation_data=(X_val_norm, y_val))


Epoch 1/170
72/72 [==============================] - 1s 3ms/step - loss: 0.3429 - mae: 0.5071 - val_loss: 0.3033 - val_mae: 0.4777
Epoch 2/170
72/72 [==============================] - 0s 2ms/step - loss: 0.3378 - mae: 0.5056 - val_loss: 0.3171 - val_mae: 0.4892
Epoch 3/170
72/72 [==============================] - 0s 1ms/step - loss: 0.3369 - mae: 0.5052 - val_loss: 0.3329 - val_mae: 0.5017
Epoch 4/170
72/72 [==============================] - 0s 1ms/step - loss: 0.3371 - mae: 0.5056 - val_loss: 0.3345 - val_mae: 0.5029
Epoch 5/170
72/72 [==============================] - 0s 1ms/step - loss: 0.3365 - mae: 0.5053 - val_loss: 0.3299 - val_mae: 0.4995
Epoch 6/170
72/72 [==============================] - 0s 2ms/step - loss: 0.3366 - mae: 0.5053 - val_loss: 0.3310 - val_mae: 0.5004
Epoch 7/170
72/72 [==============================] - 0s 1ms/step - loss: 0.3368 - mae: 0.5055 - val_loss: 0.3266 - val_mae: 0.4967
Epoch 8/170
72/72 [==============================] - 0s 1ms/step - loss: 0.3367 - m

<IPython.core.display.Javascript object>

In [23]:
# Save the model
model_1000_aug.save("models/model_standard_1000_augmented.h5")

<IPython.core.display.Javascript object>

In [24]:
# Load and Evaluate the model on the validation dataset
new_model_1000_aug = tf.keras.models.load_model(
    "models/model_standard_1000_augmented.h5"
)
val_metrics_1000_aug = new_model_1000_aug.evaluate(X_val_norm, y_val)

32/32 [==============================] - 0s 910us/step - loss: 0.3338 - mae: 0.5021


<IPython.core.display.Javascript object>

In [25]:
new_model_1000_aug.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                320       
                                                                 
 dense_10 (Dense)            (None, 48)                3120      
                                                                 
 dense_11 (Dense)            (None, 2)                 98        
                                                                 
Total params: 3,538
Trainable params: 3,538
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

### 5 Equivariant Neural Networks
Train an equivariant neural network on the original dataset. To that end, you need to implement a custom model using the Keras Subclassing API. All layers except the output layer must have a weight matrix that looks as follows
W` :=
[
[a b c b]
[b a b c]
[c b a b]
[b c b a]
]
where a; b; c are the parameters that shall be learned. Moreover, equivariant layers have no bias vectors, i.e., you have b` = 0. The output layer of an equivariant network has a weight matrix WL :=
[
[d -d -d d]
[-d -d d d]
]

with only one parameter d, and no bias unit as well. Apart from these specically structured weight matrices and the absence of bias vectors, equivariant networks look all like standard networks. However, they do not come as pre-implemented Keras layers, so you have to use the subclassing API. Once you have this, you can do anything else just as in case of the standard network. Save this model as model equivariant [m].h5.

### Hypertuning Training dataset without Augmentation

In [26]:
custom_objects = {
    "EquivariantOutputLayer": EquivariantOutputLayer,
    "EquivariantHiddenLayer": EquivariantHiddenLayer,
}

<IPython.core.display.Javascript object>

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=10)

# Define the tuner object
tuner_5 = kt.Hyperband(
    eq_build_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="equivariant_nn",
)

In [ ]:
tuner_5.search_space_summary()

In [ ]:
# Start the search
tuner_5.search(X_train_1000_norm, y_train_1000,     validation_data=(X_val_norm, y_val),
    callbacks=[early_stop], epochs=200)


In [ ]:
best_hps_enn = tuner_5.get_best_hyperparameters(num_trials=1)[0]
model_hp_enn = tuner_5.hypermodel.build(best_hps_enn)


In [ ]:
best_hps_enn.values

### Hypertuning Training dataset with Augmentation

In [27]:
early_stop = EarlyStopping(monitor="val_loss", patience=10)

# Define the tuner object
tuner_6 = kt.Hyperband(
    eq_build_model,
    objective="val_loss",
    max_epochs=200,
    factor=3,
    overwrite=True,
    directory="new_dir",
    project_name="equivariant_nn_aug",
)

# Start the search
tuner_6.search(X_train_augmented_1000, y_train_augmented_1000,     validation_data=(X_val_norm, y_val),
    callbacks=[early_stop], epochs=200)


Trial 254 Complete [00h 00m 08s]
val_loss: 0.3310951292514801

Best val_loss So Far: 0.3290695250034332
Total elapsed time: 00h 23m 00s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [28]:
best_hps_enn_aug = tuner_6.get_best_hyperparameters(num_trials=1)[0]
model_hp_enn_aug = tuner_6.hypermodel.build(best_hps_enn_aug)
# model_hp_enn_aug.values

<IPython.core.display.Javascript object>

#### Training and saving equivarant models with Hyperparamter values

##### ENN for Training with 100 samples

In [1]:
# Defining the Model
model_enn_100 = tf.keras.Sequential()
model_enn_100.add(EquivariantHiddenLayer())
model_enn_100.add(EquivariantHiddenLayer())
model_enn_100.add(EquivariantOutputLayer())

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model_enn_100.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps_enn.get("learning_rate")),
    loss="mean_squared_error",
    metrics=["mean_absolute_error"],
)
model_enn_100.fit(
    X_train_100_norm,
    y_train_100,
    epochs=best_hps_enn.get("num_epochs"),
    batch_size=best_hps_enn.get("batch_size"),
    validation_data=(X_val_norm, y_val),
    callbacks=[stop_early],
)
model_enn_100.save("models/model_equivariant_100.h5")

NameError: name 'tf' is not defined

In [ ]:
model_enn_100.summary()

In [ ]:
# load the model using the custom object scope
load_model_model_enn_100 = tf.keras.models.load_model("models/model_equivariant_100.h5",custom_objects=custom_objects)


In [ ]:
valmet_model_enn_100 = load_model_model_enn_100.evaluate(X_val_norm, y_val)

##### ENN for Training with 1000 samples

In [9]:
# Defining the Model
model_enn_1000 = tf.keras.Sequential()
model_enn_1000.add(EquivariantHiddenLayer())
model_enn_1000.add(EquivariantHiddenLayer())
model_enn_1000.add(EquivariantOutputLayer())

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model_enn_1000.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps_enn.get("learning_rate")),
    loss="mean_squared_error",
    metrics=["mean_absolute_error"],
)
model_enn_1000.fit(
    X_train_1000_norm,
    y_train_1000,
    epochs=best_hps_enn.get("num_epochs"),
    batch_size=best_hps_enn.get("batch_size"),
    validation_data=(X_val_norm, y_val),
    callbacks=[stop_early],
)
model_enn_1000.save("models/model_equivariant_1000.h5")

NameError: name 'best_hps_enn' is not defined

<IPython.core.display.Javascript object>

In [ ]:
model_enn_1000.summary()

In [ ]:
# load the model using the custom object scope
load_model_model_enn_1000 = tf.keras.models.load_model("models/model_equivariant_1000.h5",custom_objects=custom_objects)


In [ ]:
valmet_enn_1000 = load_model_model_enn_1000.evaluate(X_val_norm, y_val)

##### ENN for Training with Augmented data for 100 samples

In [29]:
# Defining the Model
model_enn_100_aug = tf.keras.Sequential()
model_enn_100_aug.add(EquivariantHiddenLayer())
model_enn_100_aug.add(EquivariantHiddenLayer())
model_enn_100_aug.add(EquivariantOutputLayer())

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model_enn_100_aug.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=best_hps_enn_aug.get("learning_rate")
    ),
    loss="mean_squared_error",
    metrics=["mean_absolute_error"],
)
model_enn_100_aug.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=best_hps_enn_aug.get("num_epochs"),
    batch_size=best_hps_enn_aug.get("batch_size"),
    validation_data=(X_val_norm, y_val),
    callbacks=[stop_early],
)
model_enn_100_aug.save("models/model_equivariant_100_augmented.h5")

Epoch 1/150
4/4 [==============================] - 1s 63ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 2/150
4/4 [==============================] - 0s 11ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 3/150
4/4 [==============================] - 0s 11ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 4/150
4/4 [==============================] - 0s 11ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 5/150
4/4 [==============================] - 0s 11ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 6/150
4/4 [==============================] - 0s 10ms/step - loss: 0.3449 - mean_absolute_error: 0.5084 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 7/150
4/4 [===========

<IPython.core.display.Javascript object>

In [30]:
model_enn_100_aug.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 equivariant_hidden_layer_4   (None, 4)                3         
 (EquivariantHiddenLayer)                                        
                                                                 
 equivariant_hidden_layer_5   (None, 4)                3         
 (EquivariantHiddenLayer)                                        
                                                                 
 equivariant_output_layer_2   (None, 2)                1         
 (EquivariantOutputLayer)                                        
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [31]:
# load the model using the custom object scope
load_model_model_enn_100_aug = tf.keras.models.load_model("models/model_equivariant_100_augmented.h5",custom_objects=custom_objects)


<IPython.core.display.Javascript object>

In [32]:
valmet_load_model_model_enn_100_aug = load_model_model_enn_100_aug.evaluate(
    X_val_norm, y_val
)

32/32 [==============================] - 0s 940us/step - loss: 0.3310 - mean_absolute_error: 0.5004


<IPython.core.display.Javascript object>

##### ENN for Training with Augmented data for 1000 samples

In [33]:
# Defining the Model
model_enn_1000_aug = tf.keras.Sequential()
model_enn_1000_aug.add(EquivariantHiddenLayer())
model_enn_1000_aug.add(EquivariantHiddenLayer())
model_enn_1000_aug.add(EquivariantOutputLayer())

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model_enn_1000_aug.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=best_hps_enn_aug.get("learning_rate")
    ),
    loss="mean_squared_error",
    metrics=["mean_absolute_error"],
)
model_enn_1000_aug.fit(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=best_hps_enn_aug.get("num_epochs"),
    batch_size=best_hps_enn_aug.get("batch_size"),
    validation_data=(X_val_norm, y_val),
    callbacks=[stop_early],
)
model_enn_1000_aug.save("models/model_equivariant_1000_augmented.h5")

Epoch 1/150
32/32 [==============================] - 1s 7ms/step - loss: 0.3362 - mean_absolute_error: 0.5052 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 2/150
32/32 [==============================] - 0s 2ms/step - loss: 0.3362 - mean_absolute_error: 0.5052 - val_loss: 0.3318 - val_mean_absolute_error: 0.5009
Epoch 3/150
32/32 [==============================] - 0s 2ms/step - loss: 0.3363 - mean_absolute_error: 0.5052 - val_loss: 0.3318 - val_mean_absolute_error: 0.5009
Epoch 4/150
32/32 [==============================] - 0s 2ms/step - loss: 0.3362 - mean_absolute_error: 0.5052 - val_loss: 0.3312 - val_mean_absolute_error: 0.5005
Epoch 5/150
32/32 [==============================] - 0s 2ms/step - loss: 0.3362 - mean_absolute_error: 0.5052 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004
Epoch 6/150
32/32 [==============================] - 0s 2ms/step - loss: 0.3362 - mean_absolute_error: 0.5052 - val_loss: 0.3311 - val_mean_absolute_error: 0.5004


<IPython.core.display.Javascript object>

In [34]:
model_enn_1000_aug.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 equivariant_hidden_layer_6   (None, 4)                3         
 (EquivariantHiddenLayer)                                        
                                                                 
 equivariant_hidden_layer_7   (None, 4)                3         
 (EquivariantHiddenLayer)                                        
                                                                 
 equivariant_output_layer_3   (None, 2)                1         
 (EquivariantOutputLayer)                                        
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [35]:
# load the model using the custom object scope
load_model_enn_1000_aug = tf.keras.models.load_model("models/model_equivariant_1000_augmented.h5",custom_objects=custom_objects)


<IPython.core.display.Javascript object>

In [36]:
valmet_model_enn_1000_aug = load_model_enn_1000_aug.evaluate(X_val_norm, y_val)

32/32 [==============================] - 0s 929us/step - loss: 0.3311 - mean_absolute_error: 0.5004


<IPython.core.display.Javascript object>

#### Check the weights a, b,c and d of the ENN model

In [ ]:
for layer in load_model_model_enn_100.layers:
    print(layer.get_config(), layer.get_weights())

In [ ]:
for layer in load_model_model_enn_1000.layers:
    print(layer.get_config(), layer.get_weights())

In [ ]:
for layer in model_enn_100_aug.layers:
    print(layer.get_config(), layer.get_weights())

In [ ]:
for layer in model_enn_1000_aug.layers:
    print(layer.get_config(), layer.get_weights())

#### Which model worked best depending on the size of the dataset?

The model that is trained for 1000 Samples without augmentation using standard model definition.

####  In which cases did data augmentation enhance the performace?

Better results are observed with more data samples using standard model definition, generated through data augmentation.